In [0]:
%restart_python

In [0]:
%sql
USE energydemand.default;

In [0]:
bronze_df = spark.table("energydemand.default.eco2mix_regional_bronze")

In [0]:
cols_to_keep = [
    "code_insee_region",
    "date",
    "heure",
    "date_heure",
    "consommation",
    "thermique",
    "nucleaire",
    "eolien",
    "solaire",
    "hydraulique",
    "bioenergies",
    "ech_physiques",
    "year",
    "month",
    "day_of_month",
    "hour_of_day",
    "minute_of_hour",
]

df_silver_clean_of_additional_cols = bronze_df.select(*cols_to_keep)

df_silver_clean_of_additional_cols.printSchema()


In [0]:
from pyspark.sql import functions as F

silver_df = (
    df_silver_clean_of_additional_cols
    # période de la journée
    .withColumn(
        "periode_journee",
        F.when(F.col("hour_of_day").between(0, 6),  "nuit")
         .when(F.col("hour_of_day").between(7, 11), "matin")
         .when(F.col("hour_of_day").between(12, 17), "après-midi")
         .otherwise("soir")
    )
    # optionnel : heures pleines (tu peux ajuster les bornes)
    .withColumn(
        "is_heures_pleines",
        F.col("hour_of_day").between(8, 20)
    )
)

In [0]:
from pyspark.sql.functions import col

silver_df = silver_df.filter(col("date_heure").isNotNull() & col("consommation").isNotNull())

silver_df.count()

In [0]:
(
    silver_df
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("energydemand.default.eco2mix_regional_silver")
)


In [0]:
silver_df = spark.table("energydemand.default.eco2mix_regional_silver")
display(silver_df)

In [0]:
silver_df.dtypes